# pc

> Point Cloud data manipulation

The indices of point cloud data `idx` is expressed as sorted `int32` array with shape `(2, n_point)`.

`idx[0,:]` is their azimuth indices and `idx[1,:]` is the range indices.

`idx` is first sorted in azimuth indices and then sorted in range indices.

Here is an example:
```
array([[0, 0, 1, 1, 2, 3],
       [2, 3, 0, 3, 1, 2]], dtype=int32)
```

In [ ]:
#| default_exp pc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
from typing import Union
try:
    import cupy as cp
    from cupy._sorting.search import _exists_kernel
except:
    pass

In [ ]:
#| export
def pc2ras(idx:Union[np.ndarray,cp.ndarray], # idx array
           pc_data:Union[np.ndarray,cp.ndarray], # data, 1D or more
           shape:tuple, # image shape
):
    '''convert sparse data to raster, filled with nan'''
    xp = cp.get_array_module(pc_data)
    raster = xp.empty((*shape,*pc_data.shape[1:]),dtype=pc_data.dtype)
    raster[:] = xp.nan
    raster[idx[0],idx[1]] = pc_data
    return raster

In [ ]:
#| hide
a = np.arange(1000,dtype=np.float32).reshape(50,20)
idx = np.arange(100).reshape(2,-1)
a_raster = pc2ras(idx,a,shape=(100,100))
np.testing.assert_array_equal(a_raster[idx[0],idx[1]],a)

In [ ]:
#| export
def _ras_dims(idx1:Union[np.ndarray,cp.ndarray], # int array, index of the first point cloud
              idx2:Union[np.ndarray,cp.ndarray], # int array, index of the second point cloud
             )->tuple: # the shape of the original raster image
    '''Get the shape of the original raster image from two index, the shape could be smaller than the truth but it doesn't matter.'''
    xp = cp.get_array_module(idx1)
    dims_az = max(int(idx1[0,-1]),int(idx2[0,-1]))+1
    dims_r = max(int(xp.max(idx1[1,:])),int(xp.max(idx2[1,:])))+1
    return (dims_az,dims_r)

In [ ]:
#| export
def pc_union(idx1:Union[np.ndarray,cp.ndarray], # int array, index of the first point cloud
             idx2:Union[np.ndarray,cp.ndarray], # int array, index of the second point cloud
            )->tuple: # the union index `idx`; index of the point in output union index that originally in the first point cloud `inv_iidx`; index of the point in output union index that only exist in the second point cloud `inv_iidx2`; index of the point in the second input index that are not in the first input point cloud
    '''Get the union of two point cloud dataset. For points at their intersection, pc_data1 rather than pc_data2 is copied to the result pc_data.'''
    # this function is modified from np.unique

    xp = cp.get_array_module(idx1)
    dims = _ras_dims(idx1,idx2)

    idx = xp.concatenate((idx1,idx2),axis=-1)
    n1 = idx1.shape[1]; n2 = idx2.shape[1]
    
    idx_1d = xp.ravel_multi_index(idx,dims=dims) # automatically the returned 1d index is in int64
    iidx = xp.argsort(idx_1d,kind='stable') # test shows argsort is faster than lexsort, that is why use ravel and unravel index
    idx_1d = idx_1d[iidx]

    inv_iidx = xp.empty_like(iidx)
    inv_iidx[iidx] = xp.arange(iidx.shape[0]) # idea taken from https://stackoverflow.com/questions/2483696/undo-or-reverse-argsort-python

    mask = xp.empty(idx_1d.shape, dtype=bool)
    mask[:1] = True
    mask[1:] = idx_1d[1:] != idx_1d[:-1]
    
    idx_1d = idx_1d[mask]
    
    _mask = mask[inv_iidx] # the mask in the original cat order
    mask1 = _mask[:n1]
    mask2 = _mask[n1:]
    
    imask = xp.cumsum(mask) - 1
    inv_iidx = xp.empty(mask.shape, dtype=np.int64)
    inv_iidx[iidx] = imask # inverse the mapping
    inv_iidx = inv_iidx[_mask]
    
    idx = xp.stack(xp.unravel_index(idx_1d,dims)).astype(idx1.dtype)
   
    return idx, inv_iidx[:n1], inv_iidx[n1:], *xp.where(mask2)

Usage:

In [ ]:
ras = cp.array([[4,3,8,3],
                [4,7,2,6],
                [9,0,3,7],
                [1,4,2,6]])
idx1 = cp.array([[0,0,1,1,2,3],
                 [2,3,0,3,1,2]],dtype=np.int32)
idx2 = cp.array([[0,0,1,2,2,3],
                 [0,3,1,1,3,1]],dtype=np.int32)
pc_data1 = cp.array([3,2,5,4,32,2])
pc_data2 = cp.array([3,5,6,2,1,4])

idx, inv_iidx1, inv_iidx2, iidx2 = pc_union(idx1,idx2)

With all the returns in `pc_union`, it is very easy to construct the union data:

In [ ]:
pc_data = cp.empty((idx.shape[1],*pc_data1.shape[1:]),dtype=pc_data1.dtype)
pc_data[inv_iidx1] = pc_data1
pc_data[inv_iidx2] = pc_data2[iidx2]

np.testing.assert_equal(cp.asnumpy(pc_data),np.array([3,3,2,5,6,4,32,1,4,2]))
np.testing.assert_equal(cp.asnumpy(ras[idx[0],idx[1]]),np.array([4,8,3,4,7,6,0,7,4,2]))

In [ ]:
#| hide
## deprecated, I think return two index is better
def _cp_intersect1d(arr1:cp.ndarray,
                    arr2:cp.ndarray,
):
    '''Copy of the cupy.intersect1d.
    assume unique
    arr1 and arr2 are assumed to be 1d unique array.
    Only return indices of arr1.
    '''
    mask = _exists_kernel(arr1, arr2, arr2.size, False)
    int1d = arr1[mask]
    arr1_indices = cp.flatnonzero(mask)
    return int1d, arr1_indices

In [ ]:
#| hide
## deprecated
start1 = 0; end1 = 14
start2 = 4; end2 = 20
arr1 = cp.arange(start1,end1,dtype=np.int64)
arr2 = cp.arange(start2,end2,dtype=np.int64)
int1d, idx = _cp_intersect1d(arr1,arr2)
np.testing.assert_almost_equal(cp.asnumpy(int1d),np.arange(start2,end1,dtype=np.int64))
np.testing.assert_almost_equal(cp.asnumpy(idx),np.arange(start2-start1,end1-start1,dtype=np.int64))

In [ ]:
#| export
def pc_intersect(idx1:Union[np.ndarray,cp.ndarray], # int array, index of the first point cloud
                 idx2:Union[np.ndarray,cp.ndarray], # int array, index of the second point cloud
                 # the intersect index `idx`,
                 # index of the point in first point cloud index that also exist in the second point cloud,
                 # index of the point in second point cloud index that also exist in the first point cloud
                )->tuple:
    '''Get the intersection of two point cloud dataset.'''
    # Here I do not write the core function by myself since cupy have a different implementation of intersect1d

    xp = cp.get_array_module(idx1)
    dims = _ras_dims(idx1,idx2)

    idx1_1d = xp.ravel_multi_index(idx1,dims=dims) # automatically the returned 1d index is in int64
    idx2_1d = xp.ravel_multi_index(idx2,dims=dims) # automatically the returned 1d index is in int64

    idx, iidx1, iidx2 = xp.intersect1d(idx1_1d,idx2_1d,assume_unique=True,return_indices=True)
    idx = xp.stack(xp.unravel_index(idx,dims)).astype(idx1.dtype)

    return idx, iidx1, iidx2

In [ ]:
ras = cp.array([[4,3,8,3],
                [4,7,2,6],
                [9,0,3,7],
                [1,4,2,6]])
idx1 = cp.array([[0,0,1,1,2,3],
                 [2,3,0,3,1,2]],dtype=np.int32)
idx2 = cp.array([[0,0,1,2,2,3],
                 [0,3,1,1,3,1]],dtype=np.int32)
pc_data1 = cp.array([3,2,5,4,32,2])
pc_data2 = cp.array([3,5,6,2,1,4])

idx, iidx1, iidx2 = pc_intersect(idx1,idx2)
pc_data_int1 = pc_data1[iidx1]
pc_data_int2 = pc_data2[iidx2]

np.testing.assert_equal(cp.asnumpy(idx),np.array([[0,2],
                                                  [3,1]]))
np.testing.assert_equal(cp.asnumpy(ras[(idx[0],idx[1])]),np.array([3,0]))
np.testing.assert_equal(cp.asnumpy(pc_data_int1),np.array([2,32]))
np.testing.assert_equal(cp.asnumpy(pc_data_int2),np.array([5,2]))

In [ ]:
#| export
def pc_diff(idx1:Union[np.ndarray,cp.ndarray], # int array, index of the first point cloud
            idx2:Union[np.ndarray,cp.ndarray], # int array, index of the second point cloud
            # the diff index `idx`,
            # index of the point in first point cloud index that do not exist in the second point cloud,
           )->tuple:
    '''Get the point cloud in `idx1` that are not in `idx2`.'''

    xp = cp.get_array_module(idx1)
    dims = _ras_dims(idx1,idx2)

    idx1_1d = xp.ravel_multi_index(idx1,dims=dims) # automatically the returned 1d index is in int64
    idx2_1d = xp.ravel_multi_index(idx2,dims=dims) # automatically the returned 1d index is in int64
    
    mask = xp.in1d(idx1_1d, idx2_1d, assume_unique=True, invert=True)
    idx = idx1_1d[mask]

    idx = xp.stack(xp.unravel_index(idx,dims)).astype(idx1.dtype)
    return idx, xp.where(mask)[0]

In [ ]:
idx1 = cp.array([[0,0,1,1,2,3],
                 [2,3,0,3,1,2]],dtype=np.int32)
idx2 = cp.array([[0,0,1,2,2,3],
                 [0,3,1,1,3,1]],dtype=np.int32)

idx, iidx1 = pc_diff(idx1,idx2)
pc_data_diff = pc_data1[iidx1]

np.testing.assert_equal(cp.asnumpy(idx),np.array([[0,1,1,3],
                                                  [2,0,3,2]]))
np.testing.assert_equal(cp.asnumpy(iidx1),np.array([0,2,3,5]))

In [ ]:
#| hide
## I think there is no need for the xor function. And it is not very easy to be implemented.
def pc_xor(idx1:Union[np.ndarray,cp.ndarray], # int array, index of the first point cloud
           idx2:Union[np.ndarray,cp.ndarray], # int array, index of the second point cloud
           # the diff index `idx`,
           # index of the point in first point cloud index that do not exist in the second point cloud,
           )->tuple:
    '''Get the point cloud exclusive-or of two point clouds.'''
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()